In [1]:
import utils
import params
import random
import os
import numpy as np
from scipy.sparse import vstack

from sklearn.neural_network import MLPRegressor

In [2]:
def loadArray(path):
    return np.load(path)

In [3]:
TRAIN_SIZE = 0.8
BATCH_SIZE = 100

In [4]:
text = utils.getFiles(params.TFIDF_DESCRIPTORS_TRAIN_PATH)
images = utils.getFiles(params.CNN_PCA_DESCRIPTORS_TRAIN_PATH)

In [5]:
images[0]

'C:\\Users\\bldr1\\Desktop\\TESIS\\DESCRIPTORS\\CNN_PCA\\TRAIN\\\\PCA_COCO_train2014_000000000009.npy'

In [6]:
params.CNN_PCA_DESCRIPTORS_TRAIN_PATH

'C:\\Users\\bldr1\\Desktop\\TESIS\\DESCRIPTORS\\CNN_PCA\\TRAIN\\\\'

In [7]:
random.seed(42)
random.shuffle(text)

In [8]:
train_text = text[:int(len(text)*TRAIN_SIZE)]
val_text = text[int(len(text)*TRAIN_SIZE):]

In [9]:
def chunks(l, n):
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

In [10]:
def buildImgName(basename, folder):
    l = len(basename)
    z = '0'*(16-l)
    return os.path.join(folder,'PCA_COCO_train2014_'+z+basename)

In [11]:
buildImgName('9.npy', params.CNN_PCA_DESCRIPTORS_TRAIN_PATH)

'C:\\Users\\bldr1\\Desktop\\TESIS\\DESCRIPTORS\\CNN_PCA\\TRAIN\\\\PCA_COCO_train2014_000000000009.npy'

In [24]:
def getBatch(ficheros_text, ficheros_img, batch_size):
    
    
    for text_batch in chunks(ficheros_text,batch_size):
        text_descriptors = []
        img_descriptors = []
        
        for file in text_batch:
            file_name = os.path.basename(file)
            descriptor_text = loadArray(file).tolist()
            for element in descriptor_text:
                text_descriptors.append(element[0])
                img_descriptors.append(loadArray(buildImgName(file_name, params.CNN_PCA_DESCRIPTORS_TRAIN_PATH))[0])
            
        yield np.asarray(text_descriptors), np.asarray(img_descriptors)

In [25]:
mlp = MLPRegressor(hidden_layer_sizes=(150,130))

In [26]:
%%time
for text_descriptors, img_descriptors in getBatch(train_text, images, BATCH_SIZE):
    a = img_descriptors
    b = text_descriptors
    break

Wall time: 660 ms


In [40]:
mlp.fit(vstack(b),a)

C:\Users\bldr1\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(150, 130), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [41]:
mlp.best_loss_

18.374414854106522

In [43]:
vstack(b)[0]

<1x369537 sparse matrix of type '<class 'numpy.float64'>'
	with 11 stored elements in Compressed Sparse Row format>

In [44]:
mlp.predict(vstack(b)[0])

array([[ -1.68042034, -34.75473863,  22.79545112, -15.71070511,
         11.21120404, -29.90201819,  25.07696654, -23.25016112,
         -3.09002932,   4.92590459,  -8.28036417, -21.04774161,
         15.91029815,  30.82095495, -14.42275104,  12.75349639,
          8.56387391,  11.1204357 , -22.07696984, -27.71972602,
        -11.71845883, -17.45132223,  -7.30551561, -15.8075969 ,
        -27.78448503,  -1.2430407 ,  29.33747225,  18.14140516,
          6.13417943,  22.91067342, -10.39703337,  -9.06146769,
          1.55993984, -11.98962646,  -1.4185545 , -13.19533385,
         11.21355638, -12.36204183,   0.9454164 ,  -8.34096638,
         13.05568973,   0.77258413,  16.71473416,  18.6132016 ,
          7.73677588,  -7.8155115 ,  -6.53082175, -16.10768354,
          3.82790964,  -2.76507317,  14.33709787, -19.50483835,
        -10.6995483 , -19.81516875, -13.00917971,   0.05926584,
          8.82795326,   6.24927719,  -8.07681915,   1.50677643,
         -4.02687078,  -2.54282165, -12.

In [45]:
import pickle

In [47]:
with open(r'C:\Users\bldr1\Desktop\TESIS\MODEL\mlp_cnn.pickle', 'wb') as handle:
    pickle.dump(mlp, handle, 2)